In [ ]:
# Setup for Ollama-based agent
import os

In [ ]:
pip install -q langchain-ollama langchain-community langchain-core requests duckduckgo-search ddgs langchain langchain-hub

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
import requests

In [ ]:
# Create a simple search tool to avoid network issues
@tool
def simple_search_tool(query: str) -> str:
    """Search for information. This provides basic knowledge for common queries."""
    # Simple knowledge base for common queries
    knowledge_base = {
        "capital of madhya pradesh": "Bhopal is the capital of Madhya Pradesh, India.",
        "madhya pradesh capital": "Bhopal is the capital of Madhya Pradesh, India.",
        "bhopal": "Bhopal is the capital city of Madhya Pradesh, India. It is known as the 'City of Lakes'.",
    }
    
    # Check for matches in our knowledge base
    query_lower = query.lower()
    for key, value in knowledge_base.items():
        if key in query_lower:
            return value
    
    # Default response for unknown queries
    return f"Search results for '{query}': This is a mock search result. In a real implementation, this would return web search results."

search_tool = simple_search_tool

In [ ]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=8195461fb8916dfe75ddc9a8ce0acc22&query={city}'

  response = requests.get(url)

  return response.json()

In [ ]:
llm = ChatOllama(
    model="llama3:latest", 
    temperature=0
)

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [ ]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


In [ ]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [ ]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [ ]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)

In [ ]:
response['output']